# **Lab 13: Home Run Derby**

In [ ]:

from pylab import *
from random import uniform
from ipywidgets import interact, fixed, Dropdown 

## Same code as lab 10, but we add widgets

In [53]:
# constants/variables 
GRAVITY = 9.81  # m/s^2
LENGTH = 100  # m
TOLERANCE = 1
LOWER_BOUND = LENGTH - TOLERANCE
UPPER_BOUND = LENGTH + TOLERANCE
WALL_HEIGHT = 8  # m

velocity: float
angle: float

# map of states
dict = {0: 'The ball hit the wall!', 1: 'Home run!!', 2: 'The ball hit the ground.'}

# function for duration of projectile motion
def t_flight(velocity, angle) -> float:
    return 2*velocity*sin(radians(angle))/GRAVITY

# function to check in which state the ball is in
def checkState(x, y, dict) -> str:
    # check if the ball hit the wall
    if (LOWER_BOUND <= x <= UPPER_BOUND) and ( 0 < y <= WALL_HEIGHT):
        print(f'x: {x:.2f}m | y: {y:.2f}m')
        return dict[0]
    # check for home run
    elif x >= LENGTH and y >= WALL_HEIGHT: 
        print(f'x: {x:.2f}m | y: {y:.2f}m')
        return dict[1]
    # ball hit the ground case
    else:
        y = 0
        if x < LENGTH:
            print(f'x: {x:.2f}m | y: {y:.2f}m')
            return dict[2]

# function to graph the motion
def graphMotion(velocity, angle):
    # figure information
    figure(figsize=(8, 6))

    # parameter vectors
    t = linspace(0, t_flight(velocity, angle), 1000)
    x = velocity*t*cos(radians(angle))
    y = velocity*t*sin(radians(angle)) - 0.5*GRAVITY*t**2

    x = [item for item in x if item <= ub]
    y = y[:len(x)]
    t = t[:len(x)]

    title(f'{checkState(x[-1], y[-1], dict)}')

    xlabel('Distance [$m$]')
    ylabel('Height [$m$]')

    # plot vertical wall at x = 100 with height 8
    plot([LENGTH, LENGTH], [0, WALL_HEIGHT], 'r-')

    # plot the trajectory
    plot(x, y)

    xlim(0,LENGTH+10)
    ylim(0,50)

    # savefig('ground.png')
    # savefig('wall.png')
    # savefig('homerun.png')

interact(graphMotion, velocity=(1., 100.), angle=(1., 89.))

interactive(children=(FloatSlider(value=50.5, description='velocity', min=1.0), FloatSlider(value=45.0, descri…

<function __main__.graphMotion(velocity, angle)>

### Extra Credit: Add a pull-down menu that enables user to select different buildings

In [72]:
class Derby:
    def __init__(self):
        # Initialize building dimensions
        self.buildingWidth = uniform(5, 15)
        self.buildingHeight = uniform(10, 30)
        self.houseWidth = uniform(8, 12)
        self.houseHeight = uniform(5, 8)
    
    def getObstacleProperties(self, obstacleType):
        """Get width and height of selected obstacle"""
        if obstacleType == 'Wall':
            return 1, WALL_HEIGHT
        elif obstacleType == 'House':
            return self.houseWidth, self.houseHeight
        else:  # Building
            return self.buildingWidth, self.buildingHeight
    
    def checkCollision(self, x, y, obstacleType):
        """Check what the ball collided with"""
        width, height = self.getObstacleProperties(obstacleType)
        xStart = LENGTH - width
        
        # Hit obstacle
        if xStart <= x <= LENGTH and 0 <= y <= height:
            return 'Broken Window!' if obstacleType in ['House', 'Building'] else 'The ball hit the wall!'
        
        # Home run    
        if x >= LENGTH and y >= height:
            return 'Home run!!'
            
        # Ground hit
        return 'The ball hit the ground.'
    
    def graphMotion(self, velocity, angle, obstacleType):
        # Calculate trajectory
        t = linspace(0, 2 * velocity * sin(radians(angle)) / GRAVITY, 100)
        x = velocity * t * cos(radians(angle))
        y = velocity * t * sin(radians(angle)) - 0.5 * GRAVITY * t**2
        
        # Create plot
        fig = figure(figsize=(8, 6))
        ax = fig.gca()
        
        # Draw obstacle
        width, height = self.getObstacleProperties(obstacleType)
        xStart = LENGTH - width
        color = {'Wall': 'gray', 'House': 'brown', 'Building': 'darkgray'}[obstacleType]
        ax.add_patch(Rectangle((xStart, 0), width, height, facecolor=color, alpha=0.5))
        
        # Plot trajectory
        plot(x, y, 'b-', label='Ball Trajectory')
        
        # Plot end point
        plot(x[-1], y[-1], 'ro', label='End Point')
        
        # Set title based on collision
        title(self.checkCollision(x[-1], y[-1], obstacleType))
        
        # Set labels and limits
        xlabel('Distance [m]')
        ylabel('Height [m]')
        xlim(0, LENGTH + 10)
        ylim(0, max(50, height + 10))
        legend()
    
    def makeInteractive(self):
        """Create the interactive widget"""
        obstacleDropdown = Dropdown(
            options=['Wall', 'House', 'Building'],
            value='Wall',
            description='Obstacle:'
        )
        
        interact(self.graphMotion, 
                velocity=(1., 100., 1.),
                angle=(1., 89., 1.),
                obstacleType=obstacleDropdown)

# Create and display the widget
derby = Derby()
derby.makeInteractive()

interactive(children=(FloatSlider(value=50.0, description='velocity', min=1.0, step=1.0), FloatSlider(value=45…